In [180]:
import pandas as pd
import numpy as np
from difflib import SequenceMatcher
from re import sub

df = pd.read_csv('PeopleVote.csv', skiprows=1)
df = df.iloc[:, :9]
df_ocr = pd.read_csv('81.csv', skiprows=1)

In [181]:
df_ocr.columns = ['no', 'id', 'fullname', 'party', 'vote']
df_ocr = df_ocr.astype(np.str_)

In [182]:
def similar(a, b):
    return SequenceMatcher(lambda x: x in ' ', a, b).ratio()

In [183]:
def fix_party(p, ps):
  if not p: return p

  for x in ps:
    sim = similar(p, str(x))
    if sim > .8:
      return x
  return p

df_ocr.party = df_ocr.party.apply(lambda x: fix_party(x.replace('พรรศ', 'พรรค').replace('พรวศ', 'พรรค').replace('พรรค', ''), df.party.unique()))

In [184]:
party_b = df_ocr.party.isin(df.party.dropna())

In [185]:
TITLE = ['ร้อยตํารวจตรี', 'พันเอก', 'ร้อยตํารวจเอก', 'พันตํารวจโท', 'พลตํารวจเอก', 'พันตํารวจเอก']
REPLACE = ('นนางสาว', 'นางสาว'), ('นษาง', 'นาง'), ('นบนาย', 'นาย'), ('บาย', 'นาย'), ('นบาย', 'นาย'), ('พสตํารวจ', 'พลตำรวจ'), ('พลตำรวจตรวี', 'พลตำรวจตรี')

def split_name(name, titles):
  name = sub('[!-~]', '', name)
  for o, n in REPLACE:
    if name.startswith(o):
      name = name.replace(o, n)

  name = sub('(ร้อย|พัน|พล)(ตํารวจ)?(ตรี|โท|เอก)', '', name).strip()

  for t in titles:
    if name.startswith(t):
      return name[len(t):]
  return name

df_ocr.fullname = df_ocr.fullname.apply(lambda x: split_name(x, df.title.unique().tolist()+TITLE))

In [186]:
def vote_encoder(vote: str):
  '''
  1 = เห็นด้วย,
  2 = ไม่เห็นด้วย,
  3 = งดออกเสียง,
  4 = ไม่ลงคะแนนเสียง, 
  5 = ไม่เข้าร่วมประชุม,
  \- = ไม่ใช่วาระการประชุม
  '''
  if vote == 'เห็นด้วย':
    return 1
  if vote == 'ไม่เห็นด้วย':
    return 2
  if vote == 'งดออกเสียง':
    return 3
  if vote == 'ไม่ลงคะแนนเสียง':
    return 4
  if vote == '-':
    return 5

  p = (similar(vote,'เห็นด้วย'), similar(vote,'ไม่เห็นด้วย'), similar(vote,'งดออกเสียง'), similar(vote, 'ไม่ลงคะแนนเสียง'))
  i = np.argmax(p)
  if p[i] < .8:
    return 5
  return i + 1

In [187]:
df_ocr.vote = df_ocr.vote.apply(vote_encoder)

In [188]:
name_b = df_ocr.fullname.apply(lambda x: x.split(' ')[0]).apply(lambda x: (x == df.name.dropna()).sum())

In [189]:
def last_name(fullname):
  sep_fn = fullname.split(' ', maxsplit=1)
  if (len(sep_fn) > 1):
    return sep_fn[-1].replace('\'', '')
  return fullname

last_name_b = df_ocr.fullname.apply(last_name).isin(df.lastname.dropna())

In [190]:
name_b.sum(),last_name_b.sum(),(name_b|last_name_b).sum(),party_b.sum()

(504, 291, 406, 470)

In [191]:
included_b = (name_b)|(last_name_b)|(party_b)
df_cleaned = df_ocr[included_b]
df_cleaned

,no,id,fullname,party,vote
6,า,237,พลภูมิ วิภัติภูมิประเทศ,เพื่อไทย,1
7,2,5360,ศิรสิทธิ์ เลิศด้วยลาภ,เพื่อไทย,1
8,2,52383,สมบัติ ศรีสรีนทร์,เพื่อไทย,1
9,ด่|,ๆ933,สุภาภรณ์ คงวุฒิปัญญา,เพื่อไทย,1
10,5,009,กฤษฎา ตับเทอดทิตย์,เพื่อไทย,2
...,...,...,...,...,...
518,471,135,พางบุญญาพร นาตะธนภัทร,พลังชาติไทย,1
519,472,296,ระวิี มาศฉนาตลส,พลังธรรมใหม่,1
520,4ๆ73,179,บิคม บุญวิเศษ,พลังปวงชนไทย,1
521,87ๆ,041,คฑาเทพ เตชะเตขชเรื่องกุล,เพื่อชาติไทย,1


In [192]:
match_row = np.full(df.name.shape, np.nan)
df_c_g = df_cleaned.groupby('party')
for i, g in df.groupby('party'):
  if i not in df_c_g.groups: continue

  tmp = df_c_g.get_group(i)
  for j, r in tmp.iterrows():
    sim = g.apply(lambda x: similar(r.fullname.replace(' ', ''), x['name'] + x['lastname']), axis=1)
    if sim.max() > .6:
      if j not in match_row:
        match_row[g.iloc[sim.argmax()].name] = j

for i, row in df_ocr[included_b&(df_ocr.index.isin(np.unique(match_row))==False)].iterrows():
  sim = df.apply(lambda x: similar(row.fullname.replace(' ', ''), x['name'] + x['lastname']), axis=1)
  if sim.max() > .7:
    match_row[sim.argmax()] = i
  else:
    print(sim.argmax(), df.loc[sim.argmax()]['name'])

df_ocr[included_b&(df_ocr.index.isin(np.unique(match_row))==False)]

,no,id,fullname,party,vote
526,nan,299,รังสิกร ทิมาตฤกะ,ภูมิใจไทย,1


In [193]:
df['ocr'] = match_row


In [208]:
merged_df

,id,title,name,lastname,party,is_mp,is_senator,is_cabinet,is_active,ocr,fullname,vote
21,22,นาย,กนก,ลิ้มตระกูล,เพื่อไทย,1.0,0.0,0.0,1,87.0,นพายกนก ลิ้มตระกูล,5
22,23,ศาสตราจารย์ ดร.,กนก,วงษ์ตระหง่าน,ประชาธิปัตย์,1.0,0.0,0.0,1,374.0,กนก วงษ์ตระหง่าน,1
23,24,นาย,กมลศักดิ์,ลีวาเมาะ,ประชาชาติ,1.0,0.0,0.0,1,474.0,กมลศักดิ์ ลสีวาเมาะ,5
25,26,นาง,กรณิศ,งามสุคนธ์รัตนา,พลังประชารัฐ,1.0,0.0,0.0,1,149.0,กรณีศ งามสุคนธ์รัตนา,1
26,27,นาย,กรวีร์,ปริศนานันทกุล,ภูมิใจไทย,1.0,0.0,0.0,1,255.0,กรวีรว์ ปริศนานันทกุส,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1053,1054,นาย,ชวน,ชูจันทร์,พลังประชารัฐ,1.0,0.0,0.0,1,232.0,ชวน ซูจันทร์,1
1054,1055,นาย,จักพันธ์,ปิยพรไพบูลย์,ประชาธิปัตย์,1.0,0.0,0.0,1,420.0,จักพันธ์ ปิยพรไพบูลย์,1
1055,1056,นางสาว,สุภาพร,กำเนิดผล,ประชาธิปัตย์,1.0,0.0,0.0,1,384.0,สุภาพร กําเน็ตผล,1
1056,1057,นางสาว,สุเนตตา,แซ่โก๊ะ,รวมพลังประชาชาติไทย,1.0,0.0,0.0,1,496.0,สุเนตตา แซ่โก๊ะ,1


In [198]:
merged_df = df.merge(df_ocr[['fullname', 'vote']], left_on='ocr', right_index=True)
merged_df.vote.values

array([5, 1, 5, 1, 1, 1, 1, 2, 1, 1, 1, 2, 5, 1, 1, 1, 5, 5, 5, 1, 2, 1,
       1, 5, 2, 1, 2, 1, 1, 1, 1, 1, 2, 5, 1, 5, 1, 1, 5, 5, 1, 2, 2, 5,
       5, 2, 1, 1, 1, 5, 5, 1, 1, 1, 2, 1, 1, 2, 5, 2, 5, 2, 2, 5, 5, 2,
       1, 1, 1, 2, 1, 5, 2, 2, 1, 2, 5, 2, 5, 5, 1, 5, 1, 1, 1, 1, 1, 1,
       2, 1, 5, 1, 5, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2,
       2, 2, 5, 2, 1, 1, 2, 1, 1, 1, 5, 1, 5, 1, 1, 1, 5, 2, 2, 2, 2, 1,
       1, 2, 2, 2, 5, 5, 2, 2, 1, 2, 1, 1, 2, 2, 1, 2, 2, 2, 1, 1, 5, 5,
       1, 1, 1, 1, 1, 2, 1, 5, 5, 2, 5, 2, 1, 1, 1, 1, 2, 5, 5, 1, 2, 1,
       2, 2, 1, 2, 1, 2, 1, 5, 1, 1, 2, 1, 2, 1, 5, 5, 2, 2, 2, 1, 1, 2,
       2, 2, 2, 2, 1, 1, 1, 1, 2, 5, 1, 2, 1, 1, 1, 5, 1, 2, 1, 1, 5, 2,
       1, 1, 1, 1, 1, 2, 5, 1, 2, 5, 1, 5, 1, 2, 1, 1, 2, 5, 1, 1, 1, 1,
       1, 1, 1, 1, 5, 1, 2, 1, 1, 1, 2, 2, 1, 1, 2, 2, 2, 1, 5, 5, 5, 1,
       1, 1, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 5, 2, 2, 1,
       1, 2, 5, 5, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 5,

In [195]:
df_ch = pd.read_csv('PeopleVote.csv', skiprows=1)

In [206]:
df_ch['votelog.__81'].replace('-', 5).fillna(0).astype(np.int8).values

array([5, 5, 5, ..., 1, 1, 1], dtype=int8)

In [205]:
merged_df[(df_ch.loc[merged_df.index]['votelog.__81'].replace('-', 5).fillna(0).astype(np.int16)==merged_df.vote)==False].sample()

,id,title,name,lastname,party,is_mp,is_senator,is_cabinet,is_active,ocr,fullname,vote
474,475,นาย,องอาจ,วงษ์ประยูร,เพื่อไทย,1.0,0.0,0.0,1,143.0,องอาจ วงษ์ประยูร,5


In [202]:
df_ch

,id,title,name,lastname,party,is_mp,is_senator,is_cabinet,is_active,votelog.__1,...,votelog.__74,votelog.__75,votelog.__76,votelog.__77,votelog.__78,votelog.__79,votelog.__80,votelog.__81,votelog.__82,Unnamed: 80
0,1,พลเอก,ประยุทธ์,จันทร์โอชา,NaN,0.0,0.0,1.0,1,-,...,-,-,-,-,-,-,-,-,-,NaN
1,2,พลเอก,ประวิตร,วงษ์สุวรรณ,พลังประชารัฐ,0.0,0.0,1.0,1,-,...,-,-,-,-,-,-,-,-,-,NaN
2,3,ดร.,สมคิด,จาตุศรีพิทักษ์,พลังประชารัฐ,0.0,0.0,1.0,0,-,...,-,-,-,-,-,-,-,-,-,NaN
3,4,ดร.,วิษณุ,เครืองาม,NaN,0.0,0.0,1.0,1,-,...,-,-,-,-,-,-,-,-,-,NaN
4,5,พลเอก,ชัยชาญ,ช้างมงคล,NaN,0.0,0.0,1.0,1,-,...,-,-,-,-,-,-,-,-,-,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1053,1054,นาย,ชวน,ชูจันทร์,พลังประชารัฐ,1.0,0.0,0.0,1,-,...,4,NaN,4,1,NaN,NaN,1,-,1,NaN
1054,1055,นาย,จักพันธ์,ปิยพรไพบูลย์,ประชาธิปัตย์,1.0,0.0,0.0,1,NaN,...,1,1,1,1,NaN,1,1,1,NaN,NaN
1055,1056,นางสาว,สุภาพร,กำเนิดผล,ประชาธิปัตย์,1.0,0.0,0.0,1,NaN,...,NaN,NaN,NaN,1,1,1,1,1,1,NaN
1056,1057,นางสาว,สุเนตตา,แซ่โก๊ะ,รวมพลังประชาชาติไทย,1.0,0.0,0.0,1,NaN,...,1,NaN,1,1,3,1,1,1,1,NaN
